In [ ]:
import pandas as pd
from tensorflow import keras
import numpy as np

In [2]:
df = pd.read_csv('Cancer_Data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.dropna(axis='columns',how='all',inplace=True)

In [4]:
target = df.diagnosis
df = df.drop(['diagnosis','id'],axis='columns')

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(df)
target = pd.get_dummies(target,drop_first=True).values

In [7]:
from sklearn.model_selection import StratifiedKFold

In [8]:
kfold = StratifiedKFold(n_splits=10)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def get_accuracy(x_train,x_test,y_train,y_test):
    model = keras.Sequential([
        keras.layers.Dense(1, input_shape=(30,), activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(
    monitor='val_loss',   
    patience=50,          
    restore_best_weights=True  
)
    model.fit(x_train,y_train,validation_data=(x_test, y_test),epochs=5000,callbacks=[early_stop], verbose=0)

    score =model.evaluate(x_test,y_test)
    return score

In [10]:
accuracy = []

for train_index,test_index in kfold.split(scaled_data,target):

    x_train = scaled_data[train_index]
    x_test = scaled_data[test_index]
    y_train = target[train_index]
    y_test = target[test_index]

    accuracy.append(get_accuracy(x_train,x_test,y_train,y_test))

c:\Users\shaik\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0419
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9825 - loss: 0.0721
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9825 - loss: 0.0877
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9649 - loss: 0.0759
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0373
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9649 - loss: 0.0783 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9298 - loss: 0.1641
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0343
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0183
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9821 - loss: 0.0617


In [18]:
acc = []

for i in range(0,10):
    acc.append(accuracy[i][1])

acc

[1.0,
 0.9824561476707458,
 0.9824561476707458,
 0.9649122953414917,
 1.0,
 0.9649122953414917,
 0.9298245906829834,
 1.0,
 1.0,
 0.9821428656578064]

In [20]:
print(f"Mean accuracy: {np.mean(acc)*100:.2f}%")
print(f"Std deviation: {np.std(acc)*100:.2f}%")

Mean accuracy: 98.07%
Std deviation: 2.14%
